# MiniGPT LLM

In [12]:
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch

from nimrod.text.tokenizers import CharTokenizer
from nimrod.text.datasets import SimpleCharDataset
from nimrod.models.transformer import TransformerBlock

from livelossplot import PlotLosses

from pathlib import Path
from dataclasses import dataclass, asdict
import time

# Data

In [13]:
text = Path('../data/text/tiny_shakespeare.txt').read_text()
print(text[:25])
tok = CharTokenizer.from_text(text)
print(f"vocabulary size: {len(tok)}")
encoded = tok.encode(text)
print(encoded[:10])
ds = SimpleCharDataset(encoded, 10)
print(f"length of data str: {len(ds)}")
n_train = int(0.8 * len(ds))
n_eval = int(0.1 * len(ds))
n_test = len(ds) - n_train - n_eval
train_ds, eval_ds, test_ds = random_split(ds, lengths=(n_train, n_eval, n_test))
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
eval_dl = DataLoader(train_ds, batch_size=32, shuffle=False)
test_dl = DataLoader(train_ds, batch_size=32, shuffle=False)

First Citizen:
Before we 
vocabulary size: 65
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])
length of data str: 1115384


## Model

In [14]:
class MiniGPT(nn.Module):
    def __init__(
        self,
        vocab_size:int,
        embed_dim:int,
        n_head:int,
        context_length:int,
        dropout:float,
        n_blocks:int
    ):

        super().__init__()
        self.context_length = context_length
        self.tok_embed = nn.Embedding(vocab_size, embed_dim)
        self.pos_embed = nn.Embedding(context_length, embed_dim)
        block_params = [embed_dim, n_head, context_length, dropout]
        self.blocks = [TransformerBlock(*block_params) for _ in range(n_blocks)]
        self.blocks = nn.Sequential(*self.blocks)
        self.ln = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, vocab_size)

    def forward(
        self,
        x:torch.LongTensor # (B,T) list of token IDS
        ):
        B,T = x.shape
        tok_embed = self.tok_embed(x) # B, T, C
        pos_embed = self.pos_embed(torch.arange(T, device=x.device)) # B, T, C
        x = tok_embed + pos_embed
        x = self.blocks(x)
        x = self.ln(x)
        logits = self.head(x)
        return logits

    def generate(
        self,
        x:torch.LongTensor, # tok ids (B, T)
        max_tokens: int # max size of sequence gen
    ):  
        self.eval()
        for i in range(max_tokens):
            if x.size(1) >= self.context_length:
                break
            with torch.no_grad():
                logits = self.forward(x) # (B, T, n_classes)
                # look at last time step only
                logits = logits[:, -1, :]
                probs = torch.softmax(logits, dim=-1)
                pred = torch.multinomial(probs, num_samples=1)
                x = torch.cat([x, pred], dim=1) #(T+1)
        return x



### Config

In [15]:
@dataclass
class GPTConfig:
    vocab_size:int
    embed_dim:int = 16
    n_head:int = 2
    context_length:int = 10
    n_blocks:int = 2
    dropout:float = 0.1


    def __post_init__(self):
        assert self.embed_dim % self.n_head == 0, "embed_dim must be divisible by n_head"
        assert self.dropout >= 0 and self.dropout <= 1, "dropout must be between 0 and 1"

cfg = GPTConfig(vocab_size=len(tok))

In [9]:
m = MiniGPT(**asdict(cfg))
print(m)

MiniGPT(
  (tok_embed): Embedding(65, 16)
  (pos_embed): Embedding(10, 16)
  (blocks): Sequential(
    (0): TransformerBlock(
      (ln1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-1): 2 x AttentionHead(
            (key): Linear(in_features=16, out_features=8, bias=False)
            (query): Linear(in_features=16, out_features=8, bias=False)
            (value): Linear(in_features=16, out_features=8, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (proj): Linear(in_features=16, out_features=16, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ln2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=16, out_features=64, bias=True)
          (1): ReLU()
          (2): Linear(in_features=64, out_features=16, bias=True)
          (3): Dropout(p=

### Usage

In [10]:
device = 'cpu'
B, T, C = 5, cfg.context_length, cfg.embed_dim
x = torch.randint(0, cfg.vocab_size, (B,T), dtype=torch.long).to(device)
m = m.to(device)
y = m(x) # B,T, n_classes
print(y.shape)

torch.Size([5, 10, 65])


In [11]:
prompt = "hello!"
encoded = tok.encode(prompt).unsqueeze(dim=0)
print(encoded)
res = m.generate(encoded, max_tokens=25)
print(res)
print(tok.decode(res.squeeze()))

tensor([[46, 43, 50, 50, 53,  2]])
tensor([[46, 43, 50, 50, 53,  2, 17, 41, 23, 48]])
hello!EcKj


## Training Loop

In [ ]:
device = 'mps'
cfg = GPTConfig(vocab_size=len(tok))
cfg.embed_dim:int = 16
cfg.n_head:int = 1
cfg.context_length:int = 10
cfg.n_blocks:int = 1

m = MiniGPT(**asdict(cfg)).to(device)

n_epochs = 2
lr = 3e-3

criterion = nn.CrossEntropyLoss()
optim = torch.optim.AdamW(m.parameters(), lr=lr)

liveplot = PlotLosses()

for epoch in range(n_epochs):
    m.train()
    train_running_loss = 0
    val_running_loss = 0
    train_loss_hist = []
    eval_loss_hist = []

    for step, batch in enumerate(train_dl):
        optim.zero_grad()
        x, y = batch
        x,y = x.to(device), y.to(device)
        B, T = x.shape
        logits = m(x) # B, T, V
        B,T,V = logits.shape
        logits = logits.view(B*T, V)
        y = y.view(B*T)
        loss = criterion(logits, y)
        train_running_loss += loss.item()
        # if not step%1000:
        #     print(loss.item())
        loss.backward()
        optim.step()
    avg_train_loss = train_running_loss / len(train_dl)
    # print(f"epoch: {epoch} train loss: {avg_train_loss}")

    for step, batch in enumerate(eval_dl):
        optim.zero_grad()
        x, y = batch[0].to(device), batch[1].to(device)
        B, T = x.shape
        logits = m(x)
        B, T, V = logits.shape
        logits = logits.view(B*T, V)
        y = y.view(B*T)
        val_loss = criterion(logits, y)
        val_running_loss += val_loss.item()
        loss.backward()
        optim.step()
    
    avg_val_loss = val_running_loss / len(eval_dl)
    print(f"epoch: {epoch} train_loss: {avg_train_loss} val_loss {avg_val_loss}")
    liveplot.update({
        'train_loss': avg_train_loss,
        'val_loss': avg_val_loss
    })
    liveplot.send()


## Test

In [3]:
prompt = "hello!"
encoded = tok.encode(prompt).unsqueeze(dim=0).to(device)
print(encoded)
res = m.generate(encoded, max_tokens=4)
print(res)
print(tok.decode(res.squeeze()))

NameError: name 'tok' is not defined